# FIS-Data Retriever

### Imports

In [6]:
import numpy as np
import pandas as pd
import json
import re
import urllib.request

from bs4 import BeautifulSoup
from datetime import datetime
from functools import lru_cache

### Constants

In [2]:
START_YEAR = 1967
END_YEAR = 2020

## Ski-DB parser

**[Website](https://www.ski-db.com/)**

**[Profiles](https://www.ski-db.com/db/profiles/beat_feuz_sui_511383.php)**, of the form `https://www.ski-db.com/db/profiles/<id>.php`

In [ ]:
def getSeasonURL(year, isMen=True):
    """Get the Ski-DB page for a particular season.
    """
    if not START_YEAR <= year <= END_YEAR:
        raise ValueError(f"An invalid WC year was given (from {START_YEAR} to {END_YEAR}).")
    gender = 'm' if isMen else 'f'
    endSeason = year % 100
    startSeason = (year-1) % 100
    return f"https://www.ski-db.com/db/{endSeason:02d}/cal_{gender}{startSeason:02d}{endSeason:02d}.php"

In [ ]:
def getRacesURLs(year, isMen=True):
    """Get all Ski-DB pages for the races of a particular season.
    """
    seasonURL = getSeasonURL(year, isMen)
    print(f"[INFO] Getting races URLs for the {year} season")
    page = urllib.request.urlopen(seasonURL)
    soup = BeautifulSoup(page)
    races = soup.findAll("tbody", {"class": "skidb"})

    raceURLs = []
    for race in races:
        raceURLs.append(race.findAll('a')[0]['href'])
    
    return raceURLs

In [ ]:
def parseRace(raceURL):
    page = urllib.request.urlopen(raceURL)
    soup = BeautifulSoup(page)
    raceDB = {}
    
    titles = soup.findAll("h1")
    if len(titles) < 2:
        print(f"[WARN] No metadata for {raceURL}")
    else:
        metadata = titles[1].text.split(' ')
        raceDB['date'] = ''.join(metadata[0].split('-')[::-1]).strip()

        locationMetadata = ' '.join(metadata[1:])
        raceDB['venue'] = locationMetadata.split('[')[0].strip()
        raceDB['country'] = locationMetadata.split('-')[1].strip().split('\n')[0].strip()
        raceDB['event'] = locationMetadata.split('\n')[3].strip()
    
    skiers = soup.findAll("tbody", {"class": "skidb"})[0].findAll("tr", {"class": ["blanc", "alt"]})
    raceDB['results'] = {}
    for idx, skier in enumerate(skiers):
        info = skier.findAll("td")
        skierDB = {}
        skierDB['rank'] = info[0].text.strip()
        skierDB['name'] = info[1].text.strip()
        skierDB['country'] = info[2].text.strip()
        skierDB['data'] = [i.text.strip() for i in info[3:] if i]
        skierDB['id'] = skier.findAll("a")[0]['href'].split('/')[-1].split('.')[0]
        raceDB['results'][idx] = skierDB
    
    return raceDB

In [ ]:
def saveJSON(data, filename):
    with open(f'../data/{filename}.json', 'w+') as f:
        json.dump(data, f)

In [ ]:
def parseSkiDB(isMen=True):
    db = {}

    for year in range(START_YEAR, END_YEAR+1):
        yearDB = {}
        raceURLs = getRacesURLs(year, isMen)
        
        for idx, raceURL in enumerate(raceURLs):
            yearDB[idx] = parseRace(raceURL)
        db[year] = yearDB
        saveJSON(db, "wcm" if isMen else "wcf")
    
    return db

### Men's Parsing

In [ ]:
db_m = parseSkiDB(True)

**Error handling**

In [ ]:
# https://www.ski-db.com/db/87/cal_m8687.php
berlinYear = 1987
berlinRace = 13

db_m[berlinYear][berlinRace]['date']   = "19861228"
db_m[berlinYear][berlinRace]['venue']  = "Berlin"
db_m[berlinYear][berlinRace]['country'] = "GER"
db_m[berlinYear][berlinRace]['event']  = "Parallel"

saveJSON(db_m, "wcm")

NB:

- Kitzbüehl present 2 times in 19xx on 12/01/199** (?), date parsed wrongly
- Patrick Thaler disqualifed during Adelboden Giant Slalom 1999 (1999-01-12), not 24th

### Women's Parsing

In [ ]:
db_f = parseSkiDB(False)

**Error handling**

- False time for 2nd run of Elisabeth Goergl, 2010-01-16, maribor, GS

In [ ]:
#https://www.ski-db.com/db/83/918308wc.php
tremblantYear = 1983
tremblantRace = 22

db_f[tremblantYear][tremblantRace]['date']    = "19830305"
db_f[tremblantYear][tremblantRace]['venue']   = "Mont Tremblant"
db_f[tremblantYear][tremblantRace]['country'] = "CAN"
db_f[tremblantYear][tremblantRace]['event']   = "Downhill"

# https://www.ski-db.com/db/83/938303wc.php
tremblantRace = 23

db_f[tremblantYear][tremblantRace]['date']    = "19830306"
db_f[tremblantYear][tremblantRace]['venue']   = "Mont Tremblant"
db_f[tremblantYear][tremblantRace]['country'] = "CAN"
db_f[tremblantYear][tremblantRace]['event']   = "Giant Slalom"

## The following have weird errors due to an error in the HTML
# https://www.ski-db.com/db/09/91200903wc.php
data = db_f['2009']['25']['results']['25']['data']
data = data[:3]
data[1] = data[1].split(' ')[0]
data[2] = ''
db_f['2009']['25']['results']['25']['data'] = data

# https://www.ski-db.com/db/10/91201006wc.php
data = db_f['2010']['24']['results']['27']['data']
data = data[:3]
data[1] = data[1].split(' ')[0]
data[2] = ''
db_f['2010']['24']['results']['27']['data'] = data

saveJSON(db_f, "wcf")

## Data processing

1967-2003: No ski brands

2003-2020: Ski brands as last index or data

In [ ]:
with open("../data/wcm.json") as f:
    db_m = json.load(f)
with open("../data/wcf.json") as f:
    db_f = json.load(f)

In [ ]:
def addLine(db, season, date, venue, country, event,
            ath_rank=0, ath_name='', ath_country='',
            ath_time_run_1=0, ath_time_run_2=0,
            ath_time=0, ath_time_diff=0, ath_ski='', ath_id=''):
    db["season"].append(season)
    db["date"].append(date)
    db["venue"].append(venue)
    db["country"].append(country)
    db["event"].append(event)
    db["ath_rank"].append(ath_rank)
    db["ath_name"].append(ath_name)
    db["ath_country"].append(ath_country)
    db["ath_time_run_1"].append(ath_time_run_1)
    db["ath_time_run_2"].append(ath_time_run_2)
    db["ath_time"].append(ath_time)
    db["ath_time_diff"].append(ath_time_diff)
    db["ath_ski"].append(ath_ski)
    db["ath_id"].append(ath_id)
    return db

In [ ]:
def convertTimings(toParse):
    """Convert all times in cs (hundredths of second).
    """
    if not toParse or 'n/d' in toParse:
        return 0
    timing = toParse.split('.')
    time = int(timing[1][:2])
    timing = timing[0].split("'")
    if len(timing) > 1:
        time += int(timing[0])*6000
        time += int(timing[1])*100
    else:
        time += int(timing[0])*100
    return int(time)

In [ ]:
def parseDB(db):
    db_parsed = {
        "season": [],
        "date": [],
        "venue": [],
        "country": [],
        "event": [],
        "ath_rank": [],
        "ath_name": [],
        "ath_country": [],
        "ath_time_run_1": [],
        "ath_time_run_2": [],
        "ath_time": [],
        "ath_time_diff": [],
        "ath_ski": [],
        "ath_id": [],
    }
    for year in range(START_YEAR, END_YEAR+1):
        db_season = db[str(year)]
        season = year
        for db_race in db_season.values():
            date = datetime.strptime(db_race['date'], "%Y%m%d")
            venue = db_race['venue']
            country = db_race['country']
            event = db_race['event']
            results = db_race['results']
            if results:
                for athlete in results.values():
                    rank = athlete['rank']
                    if rank.isdigit():
                        rank = int(rank)
                    elif rank[0] == '.':
                        rank = int(rank[1:])
                    name = athlete['name']
                    ath_country = athlete['country']
                    data = athlete['data']
                    
                    ath_time = data[2] if len(data) > 3 else data[0] if len(data) > 1 else None
                    ath_time_diff = data[3] if len(data) > 3 else data[1] if len(data) > 1 else None
                    if ath_time_diff and ath_time_diff[1:].replace('.', '').isdigit():
                        ath_time_diff = int(ath_time_diff.replace('.', ''))
                    elif ath_time_diff and ath_time_diff[:2] == '+-':
                        # Fix weird bug in Ski-DB
                        ath_time_diff = int(ath_time_diff[2:].replace('.', ''))
                    else:
                        ath_time_diff = 0
                    ath_time_run_1 = data[0] if len(data) > 3 else None
                    ath_time_run_2 = data[1] if len(data) > 3 else None
                    ath_ski = ''
                    if len(data) == 5:
                        ath_ski = data[4]
                    elif len(data) == 3:
                        ath_ski = data[2]
                    db_parsed = addLine(db_parsed, season, date, venue, country, event,
                                        rank, name, ath_country, convertTimings(ath_time_run_1), convertTimings(ath_time_run_2),
                                        convertTimings(ath_time), ath_time_diff, ath_ski, athlete['id'])
            else:
                db_parsed = addLine(db_parsed, season, date, venue, country, event)
    return db_parsed

In [ ]:
def convertToDF(db):
    df = pd.DataFrame(parseDB(db))
    df.ath_time_run_1 = df.ath_time_run_1.astype(int)
    return df

In [ ]:
dfm = convertToDF(db_m)
dfm.to_csv('../data/wcm.csv', index=False)
dfm.head()

In [ ]:
dff = convertToDF(db_f)
dff.to_csv('../data/wcf.csv', index=False)
dff.head()

## Analysis

Note that all times are recorded as hundredths of seconds, as to not lose any precision with floating point numbers.

To discuss: ranks are not integers beacause they contain information on DNFs, etc. Should we convert them  to integers and hold the extra information in another column?

In [ ]:
def parseCSV(isMale=True):
    gender = 'm' if isMale else 'f'
    df = pd.read_csv(f'../data/wc{gender}.csv')
    df = df.replace(np.nan, '', regex=True)
    df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
    return df

In [ ]:
dfm = parseCSV()
dff = parseCSV(False)

In [ ]:
dfm['ath_rank'].value_counts().head(10)

In [ ]:
dff[(dff.)]

## Athlete photos

In [71]:
@lru_cache(10000)
def getSearchSoup(firstname, lastname, gender):
    g = 'M' if gender else 'F'
    url = f"https://www.fis-ski.com/DB/general/biographies.html?lastname={lastname}&firstname={firstname}&sectorcode=AL&gendercode={g}&search=true"
    page = urllib.request.urlopen(url)
    return BeautifulSoup(page)

In [93]:
def getAthleteInfo(firstname, lastname, gender):
    try:
        soup = getSearchSoup(firstname, lastname, gender)
    except:
        print(f"[ERR] Could not parse {firstname} {lastname}")
        return None, 'ENC'
    entries = soup.findAll("a", {"class": "table-row"})
    if len(entries) == 0:
        return None, 'ERR'
    if len(entries) > 1:
        return None, 'WAR'
    info = entries[0].find('div').findAll('div')
    return info[3].find(text=True), info[8].find(text=True)

In [81]:
def gatherNames(gender):
    g = 'm' if gender else 'f'
    wc = pd.read_csv(f'../../data/wc{g}.csv')
    names = list(set(wc.ath_name.tolist()))[1:]
    searchNames = {}
    for n in names:
        split_n = n.strip().split(' ')
        searchNames[n] = split_n[0], split_n[-1]
    return searchNames

In [84]:
def gatherInfo(gender):
    searchNames = gatherNames(gender)
    found = {}
    error = {}
    for name, (fn, ln) in searchNamesM.items():
        id, bd = getAthleteInfo(fn, ln, True)   
        if id:
            found[name] = id, bd
        else:
            error[name] = id, bd
    return found, error

In [94]:
foundMale, errorMale = gatherInfo(True)

[ERR] Could not parse HÅkon Mjoen


In [101]:
print(f'ERR: {sum([1 for name, (a, err) in errorMale.items() if err == "ERR"])}')
print(f'WAR: {sum([1 for name, (a, err) in errorMale.items() if err == "ERR"])}')
print(f'ENC: {sum([1 for name, (a, err) in errorMale.items() if err == "ENC"])}')
#for name, (a, err) in errorMale:
#    if err == "WAR"

ERR: 170
WAR: 170
ENC: 1


In [111]:
ath_db = {
    'names': [],
    'ids': [],
    'bds': [],
}
for name, (id, bd) in foundMale.items():
    ath_db['names'].append(name)
    ath_db['ids'].append(id)
    ath_db['bds'].append(bd)
for name, (id, bd) in errorMale.items():
    ath_db['names'].append(name)
    ath_db['ids'].append(id)
    ath_db['bds'].append(bd)

In [114]:
pd.DataFrame(ath_db).to_csv('ath_male.csv', index=False)